<a href="https://colab.research.google.com/github/mirceaachirili/youtube-summarize/blob/main/summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install git+https://github.com/openai/whisper.git 
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-h8_jmyn7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-h8_jmyn7
  Resolved https://github.com/openai/whisper.git to commit f82bc59f5ea234d4b97fb2860842ed38519f7e65
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import packages 
import whisper
import pytube as pt
from transformers import pipeline

In [ ]:
model = whisper.load_model("base.en")

summarizer = pipeline('summarization', model='philschmid/bart-large-cnn-samsum')

100%|███████████████████████████████████████| 139M/139M [00:08<00:00, 17.0MiB/s]


Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# Download audio from youtube video
yt = pt.YouTube(input())

stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio_english.mp3")

https://www.youtube.com/watch?v=RzkD_rTEBYs


'/content/audio_english.mp3'

In [ ]:
# Transcribe audio to text
result = model.transcribe("audio_english.mp3")
yt_text = result['text']

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
# Preview text
yt_text

" In the coming years, artificial intelligence is probably going to change your life, and likely the entire world. But people have a hard time agreeing on exactly how. The following are excerpts from a world economic forum interview where renowned computer science professor and AI expert Stuart Russell helped separate the sense from the nonsense. There's a big difference between asking a human to do something and giving that as the objective to an AI system. When you ask a human to fetch you a cup of coffee, you don't mean this should be their life's mission and nothing else in the universe matters. Even if they have to kill everybody else in Starbucks to get you the coffee before it closes, they should do that. No, that's not what you mean. What you mean. All the other things that we usually care about, they should factor into your behavior as well. The way we build AI systems now is we give them a fixed objective. The algorithms require us to specify everything in the objective. If y

In [ ]:
# Replace end of sentence characters
yt_text = yt_text.replace('.', '.<eos>')
yt_text = yt_text.replace('?', '?<eos>')
yt_text = yt_text.replace('!', '!<eos>')


In [ ]:
# Split text into chunks of maximum 500 words 
sentences = yt_text.split('<eos>')

split_limit = 500

current_split = 0
splits = []

for sentence in sentences:
  if len(splits) == current_split + 1:
    
    if len(splits[current_split]) + len(sentence.split(' ')) <= split_limit:
      splits[current_split].extend(sentence.split(' '))
    else:
      current_split += 1
      splits.append(sentence.split(' '))
  
  else:
    print(current_split)
    splits.append(sentence.split(' '))

for split_id in range(len(splits)):
  splits[split_id] = ' '.join(splits[split_id])


0


In [ ]:
# Review split count
len(splits)

3

In [ ]:
# Preview first split
splits[0]

" In the coming years, artificial intelligence is probably going to change your life, and likely the entire world.  But people have a hard time agreeing on exactly how.  The following are excerpts from a world economic forum interview where renowned computer science professor and AI expert Stuart Russell helped separate the sense from the nonsense.  There's a big difference between asking a human to do something and giving that as the objective to an AI system.  When you ask a human to fetch you a cup of coffee, you don't mean this should be their life's mission and nothing else in the universe matters.  Even if they have to kill everybody else in Starbucks to get you the coffee before it closes, they should do that.  No, that's not what you mean.  What you mean.  All the other things that we usually care about, they should factor into your behavior as well.  The way we build AI systems now is we give them a fixed objective.  The algorithms require us to specify everything in the objec

In [ ]:
# Summarize text 
summarized = summarizer(splits, max_length=120, min_length=30, do_sample=False)

In [ ]:
# Join chunks of summarized text
text = ' '.join([summary['summary_text'] for summary in summarized])

In [ ]:
# View final summarized text
text

"Stuart Russell is a renowned computer science professor and AI expert. He explains the difference between asking a human to do something and giving that as the objective to an AI system. AI systems have to know the full objective. When AI systems are unsure about their objective, they behave in a psychopathic way. Aristotle suggests that if we had fully automated weaving machines and lecturers that could pluck the lyre and use music without any humans, then we wouldn't need any workers. People think that if the machine does the work, then they will be unemployed.  By the end of the century, we're likely to have general purpose AI. The median date is around 2045. John McCarthy, who was one of the founders of AI, says it's somewhere between 5 and 500 years. "